# Классификация комментариев
**Описание проекта:** пользователи интернет-магазина могут редактировать и дополнять описания товаров, как в вики-сообществах, и комментировать правки других клиентов. Магазин хочет получить инструмент, который будет выявлять "токсичные" правки и комментарии и отправлять их на модерацию.

**Цель проекта:** построить модель классификации комментариев пользователей. Значение метрики качества *F1 (f1-мера)* на тестовой выборке должно быть не меньше 0.75. 

**Используемые данные:** набор комментариев пользователей:
* признак `text` содержит текст комментария
* целевой признак — `toxic`, где `1` — комментарий **"токсичный"**, а `0` — **"нетоксичный"**.

## Оглавление
1. [Подготовка данных](#1)
2. [Обучение и проверка моделей](#2)
3. [Выводы](#3)

<a id="1"></a>
## 1. Подготовка данных
Импортируем необходимые библиотеки и загружаем стоп-слова.

In [1]:
from nltk.corpus import stopwords 
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer 
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC
import pandas as pd
import re


stop_words = set(stopwords.words("english"))

Считаем файл с данными и убедимся, что проблем нет.

In [2]:
data = pd.read_csv("/datasets/toxic_comments.csv")
data.sample(10)

,text,toxic
147518,"Image size \n\nKindly go to Nathan Altman, Mar...",0
157854,"""\n\n You might be interested in this Locked A...",0
1482,"Yes, you are certainly predisposed to support ...",0
131342,Talk:FETCH! with Ruff Ruffman \n\nPremiere of...,0
74124,"Yeah...unfortunately, I'm too lazy to make mys...",0
145040,""" 2014 (UTC)\nSo you like to review articles f...",0
114185,"""\nOf those listed above, I'm only not comfort...",0
7842,This subpage was created during my Training Wo...,0
23472,"""\n\nActually, Wikipedia:Removing warnings was...",0
126628,Eupator is right; the article is leading towar...,0


Напишем функцию `get_lemmas` для получения лемм, применим её к столбцу `text`, а затем выделим целевой признак. 

In [3]:
def get_lemmas(text):
    lemmitizer = WordNetLemmatizer()
    text_lower = [re.sub(r"[^a-z']", "", word.lower()) for word in text.split()]
    text_lower = [word for word in text_lower]
    lemm_text = [lemmitizer.lemmatize(word) for word in text_lower]
    return " ".join(lemm_text)

In [4]:
data["lemm_text"] = data["text"].apply(get_lemmas)

In [5]:
X = data["lemm_text"]
y = data["toxic"]

Разделим данные на обучающую и тестовую выборки.

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=0.25)

Создадим объект класса `TfidfVectorizer` для подсчёта TF-IDF в дальнейшем. 

In [7]:
tfidf = TfidfVectorizer(stop_words=stop_words)

Можно переходить к обучению моделей.

<a id="2"></a>
## 2. Обучение и проверка моделей
Поскольку рассматриваемая задача является задачей бинарной классификации, будем рассматривать модель логистической регрессии (`LogisticRegression`) и метод опорных векторов (`LinearSVC`). Подбор гиперпараметров будем производить с использованием поиска по решётке и перекрёстной проверки (`GridSearchCV`). Зададим сетки гиперпараметров, используем TF-IDF-преобразование, а затем запустим обучение моделей.

In [8]:
logreg_clf = LogisticRegression(class_weight="balanced", random_state=42, max_iter=300)

svc_clf = LinearSVC(class_weight="balanced", random_state=42)

params_logreg = {"classifier__C": range(4, 8),
                "classifier": [logreg_clf]}

params_svc = {"classifier__C": [0.8, 0.9, 1.0, 1.1, 1.2],
             "classifier": [svc_clf]}

pipeline = Pipeline([("tfidfvectorizer", tfidf),
                     ("classifier", logreg_clf)])

params = [params_logreg, params_svc]

In [9]:
grid_search = GridSearchCV(pipeline, params, scoring="f1").fit(X_train, y_train)

In [10]:
print("Лучшая модель и использованные гиперпараметры:")
grid_search.best_params_

Лучшая модель и использованные гиперпараметры:


{'classifier': LogisticRegression(C=7, class_weight='balanced', max_iter=300, random_state=42),
 'classifier__C': 7}

Посмотрим на качество моделей на валидации и среднее время обучения модели (в секундах).

In [11]:
pd.DataFrame(grid_search.cv_results_)[["mean_fit_time", "mean_test_score", "params"]]

,mean_fit_time,mean_test_score,params
0,96.978245,0.758090,"{'classifier': LogisticRegression(C=7, class_w..."
1,98.274191,0.759414,"{'classifier': LogisticRegression(C=7, class_w..."
2,98.550718,0.759398,"{'classifier': LogisticRegression(C=7, class_w..."
3,116.704600,0.759721,"{'classifier': LogisticRegression(C=7, class_w..."
4,7.708286,0.754815,{'classifier': LinearSVC(class_weight='balance...
5,6.586977,0.754625,{'classifier': LinearSVC(class_weight='balance...
6,6.874841,0.753933,{'classifier': LinearSVC(class_weight='balance...
7,7.006157,0.754246,{'classifier': LinearSVC(class_weight='balance...
8,7.190679,0.753872,{'classifier': LinearSVC(class_weight='balance...


Значения *f1-меры* оказалось мало отличаются, однако у логистической регрессии значение метрики немного больше, поэтому на тестовой выборке будем использовать её. Стоит отметить, что логистическая регрессия обучается гораздо дольше, чем SVC. Учитывая это, возможно, имеет смысл использовать SVC, если есть необходимость быстро обучать модель и получать предсказания на тестовой выборке.

Рассматривать случайный лес не будем, поскольку на разреженных данных такие модели работают не очень хорошо, а время на их обучение будет больше, чем для использованных выше моделей. 

Сделаем предсказания для лучшей модели и выведем значение *f1-меры* на тестовой выборке.

In [12]:
print(f"F1-score (test): {f1_score(y_test, grid_search.predict(X_test)):.4f}")

F1-score (test): 0.7638


<a id="3"></a>
## 3. Выводы
Оба рассмотренных класса моделей, `LogisticRegression` и `LinearSVC` показали приблизительно одинаковое качество на перекрёстной проверке. При этом `LogisticRegression` обучается гораздо дольше, чем `LinearSVC`. При незначительном отличии в *f1-мере*, возможно, рациональнее будет использовать более быструю модель.